In [185]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import torch
import torch.nn as nn
import os
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.models as models
from collections import defaultdict


In [3]:
import pandas as pd

# Replace 'your_file.tsv' with the path to your .tsv file
df = pd.read_csv('./data/filtered_results.tsv', sep='\t')

# Display the first few rows
print(df.head())


/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_11977/2273208280.py:4: DtypeWarning: Columns (40,44,46,50,54,58,94,100,104,110,118,124,126,128,130,132,134,136,140,142,146,148,150,152,154,156,158,178,186,192,198,202,214,218,230,236,238,240,244,246,248,250,252,254,256,258,260,262,264,266,270,272,274,276,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,324,326,332,334,336,338,340,342,344,346,348,352,366,374,376,380,382,384,390,392,394,396,400,406,408,410,412,414,416,418,428,430,442,446,448,456,460,462,466,468,470,486,504,512,516,518,524,526,528,530,540,542,544,546,554,556,558,560,564,578,594,608,610,612,614,620,624,626,628,630,632,634,642,644,646,648,652,654,656,658,660,662,664,666,676,678,680,686,688,694,696,698,700,702,704) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/filtered_results.tsv', sep='\t')


   rowid  ccms_row_id                   Peptidoform  \
0      1            1              .SPLFM+15.995GK.   
1      2            2     .EPQVYTLPPSREEM+15.995TK.   
2      3            3         .AVM+15.995DDFAAFVEK.   
3      4            4  .EFNAETFTFHADIC-33.988TLSEK.   
4      5            5   .M+15.995ADEAGSEADHEGTHSTK.   

               Peptidoform ID      Unmod peptidoform  Total  \
0              SPLFM+15.995GK              .SPLFMGK.  11679   
1     EPQVYTLPPSREEM+15.995TK     .EPQVYTLPPSREEMTK.  11411   
2         AVM+15.995DDFAAFVEK         .AVMDDFAAFVEK.  10949   
3  EFNAETFTFHADIC-33.988TLSEK  .EFNAETFTFHADICTLSEK.  10198   
4   M+15.995ADEAGSEADHEGTHSTK   .MADEAGSEADHEGTHSTK.   9608   

   Total- Unmodified sequence  Peptidoforms- Unmodified sequence  \
0                       11681                                  2   
1                       13133                                 46   
2                       18495                                 81   
3                 

In [4]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M2_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3_unmod', 'id'],
      dtype='object', length=707)


In [5]:
pattern = r'unmod'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
intensity_cols

['Outlier groups- unmod',
 'Outlier group ratio- unmod',
 '_dyn_#Patient_01.Timepoint_1_unmod',
 '_dyn_#Patient_01.Timepoint_2_unmod',
 '_dyn_#Patient_01.Timepoint_3_unmod',
 '_dyn_#Patient_01.Timepoint_4_unmod',
 '_dyn_#Patient_01.Timepoint_5_unmod',
 '_dyn_#Patient_01.Timepoint_6_unmod',
 '_dyn_#Patient_01.Timepoint_7_unmod',
 '_dyn_#Patient_02.Timepoint_1_unmod',
 '_dyn_#Patient_02.Timepoint_2_unmod',
 '_dyn_#Patient_02.Timepoint_3_unmod',
 '_dyn_#Patient_02.Timepoint_4_unmod',
 '_dyn_#Patient_02.Timepoint_5_unmod',
 '_dyn_#Patient_02.Timepoint_6_unmod',
 '_dyn_#Patient_02.Timepoint_7_unmod',
 '_dyn_#Patient_03.Timepoint_1_unmod',
 '_dyn_#Patient_03.Timepoint_2_unmod',
 '_dyn_#Patient_03.Timepoint_3_unmod',
 '_dyn_#Patient_03.Timepoint_4_unmod',
 '_dyn_#Patient_03.Timepoint_5_unmod',
 '_dyn_#Patient_03.Timepoint_6_unmod',
 '_dyn_#Patient_03.Timepoint_7_unmod',
 '_dyn_#Patient_05.Timepoint_1_unmod',
 '_dyn_#Patient_05.Timepoint_2_unmod',
 '_dyn_#Patient_05.Timepoint_3_unmod',
 '_dyn_

In [6]:
df.drop(intensity_cols, axis=1, inplace=True)

In [7]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=369)


In [8]:
df.drop(['rowid', 'ccms_row_id', 'Peptidoform ID', 'Mass'],  axis=1, inplace=True)

In [9]:
df.columns

Index(['Peptidoform', 'Unmod peptidoform', 'Total',
       'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence',
       'Proteins', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=365)

In [10]:
df.drop(['Unmod peptidoform', 'Unmod peptidoform', 'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy'],  axis=1, inplace=True)

In [11]:
for i in df.columns:
    print(i)

Peptidoform
Total
Proteins
Lorikeet input
Orig cluster FDR
Pep Prefix
Annotation
Annotation without position
Known
Num mod frags
PValue
% Explained
Rep cluster task
Rep cluster user
Rep cluster index
Num tasks
Rep spectrum filename
Rep spectrum scan
Outlier groups
Outlier group ratio
Unmod_Peptidoform
_dyn_#Patient_01.Timepoint_1
_dyn_#Patient_01.Timepoint_2
_dyn_#Patient_01.Timepoint_3
_dyn_#Patient_01.Timepoint_4
_dyn_#Patient_01.Timepoint_5
_dyn_#Patient_01.Timepoint_6
_dyn_#Patient_01.Timepoint_7
_dyn_#Patient_02.Timepoint_1
_dyn_#Patient_02.Timepoint_2
_dyn_#Patient_02.Timepoint_3
_dyn_#Patient_02.Timepoint_4
_dyn_#Patient_02.Timepoint_5
_dyn_#Patient_02.Timepoint_6
_dyn_#Patient_02.Timepoint_7
_dyn_#Patient_03.Timepoint_1
_dyn_#Patient_03.Timepoint_2
_dyn_#Patient_03.Timepoint_3
_dyn_#Patient_03.Timepoint_4
_dyn_#Patient_03.Timepoint_5
_dyn_#Patient_03.Timepoint_6
_dyn_#Patient_03.Timepoint_7
_dyn_#Patient_05.Timepoint_1
_dyn_#Patient_05.Timepoint_2
_dyn_#Patient_05.Timepoint_3
_

In [12]:
df.drop(['Total', 'Lorikeet input', 'Orig cluster FDR', 'Pep Prefix',
         'Annotation', 'Annotation without position', 'Known', 
         'Num mod frags', 'PValue',
'Num tasks'],  axis=1, inplace=True)

In [13]:
df.drop([
    "% Explained",
"Rep cluster task",
"Rep cluster user",
"Rep cluster index",
"Rep spectrum filename",
"Rep spectrum scan",
"Outlier groups",
"Outlier group ratio",
"Unmod_Peptidoform"
],  axis=1, inplace=True)

In [14]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_M1_healthyMale.Timepoint_1,_dyn_#Patient_M1_healthyMale.Timepoint_2,_dyn_#Patient_M1_healthyMale.Timepoint_3,_dyn_#Patient_M2_healthyMale.Timepoint_1,_dyn_#Patient_M2_healthyMale.Timepoint_2,_dyn_#Patient_M2_healthyMale.Timepoint_3,_dyn_#Patient_M3_healthyMale.Timepoint_1,_dyn_#Patient_M3_healthyMale.Timepoint_2,_dyn_#Patient_M3_healthyMale.Timepoint_3,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,NaN,"21,822,000","25,470,000",NaN,NaN,NaN,NaN,NaN,NaN,0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,NaN,"68,957,000","188,300,000",NaN,NaN,NaN,NaN,NaN,NaN,2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"287,560,000","617,050","339,430,000",NaN,NaN,NaN,NaN,NaN,NaN,4


In [15]:
pattern = r'healthy'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
print(intensity_cols)
df.drop(intensity_cols, axis=1, inplace=True)

['_dyn_#Patient_F1_healthyFemale.Timepoint_1', '_dyn_#Patient_F1_healthyFemale.Timepoint_2', '_dyn_#Patient_F1_healthyFemale.Timepoint_3', '_dyn_#Patient_F2_healthyFemale.Timepoint_1', '_dyn_#Patient_F2_healthyFemale.Timepoint_2', '_dyn_#Patient_F2_healthyFemale.Timepoint_3', '_dyn_#Patient_F3_healthyFemale.Timepoint_1', '_dyn_#Patient_F3_healthyFemale.Timepoint_2', '_dyn_#Patient_F3_healthyFemale.Timepoint_3', '_dyn_#Patient_M1_healthyMale.Timepoint_1', '_dyn_#Patient_M1_healthyMale.Timepoint_2', '_dyn_#Patient_M1_healthyMale.Timepoint_3', '_dyn_#Patient_M2_healthyMale.Timepoint_1', '_dyn_#Patient_M2_healthyMale.Timepoint_2', '_dyn_#Patient_M2_healthyMale.Timepoint_3', '_dyn_#Patient_M3_healthyMale.Timepoint_1', '_dyn_#Patient_M3_healthyMale.Timepoint_2', '_dyn_#Patient_M3_healthyMale.Timepoint_3']


In [16]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4


In [30]:
grouped = df.groupby('Proteins')


In [32]:
print(len(grouped))

1308


In [45]:
grouped.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40836,.-11.98NMFWYQQK.,tr|A0A075B6I3|A0A075B6I3_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40836
40841,.+119.037RLEEIMK.,sp|Q14244-2|MAP7_HUMAN;sp|Q14244-3|MAP7_HUMAN;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40841
40855,.+113.91DGAMFFHWRRAAEEGK.,sp|Q9NPF5|DMAP1_HUMAN;tr|Q5TG36|Q5TG36_HUMAN;t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40855
40897,.-1.01HHHHHHHH.,sp|O00257|CBX4_HUMAN;sp|O00555-2|CAC1A_HUMAN;s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40897


In [38]:
grouped.get_group('sp|P01834|IGKC_HUMAN').head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
16,.VYAC+305.068EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,289,050,000","1,791,350,000","1,438,220,000","796,622,500","1,897,062,500","1,136,975,000","1,877,952,500","2,878,650,000",...,"2,887,370,000","4,133,000,000","3,874,425,000","4,114,350,000","4,016,050,000","3,517,400,000","4,720,800,000","2,276,950,000","4,766,225,000",16
41,.VYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,174,875,000","1,385,347,500","1,348,430,000","748,765,000","1,715,380,000","2,071,575,000","1,642,750,000","2,555,525,000",...,"2,788,550,000","3,932,850,000","2,735,075,000","4,045,800,000","4,886,500,000","3,119,450,000","4,815,350,000","3,613,475,000","4,590,950,000",41
46,.VYAC+247.10EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46
73,.HKVYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"114,430,250","60,926,250","55,620,750","58,679,750","132,802,000","165,390,000","76,680,750","182,036,500",...,"186,904,250","416,370,000","184,670,000","606,580,000","811,180,000","155,809,000","520,267,500","317,365,000","422,680,000",73
94,.VDNALQSGNSQE+21.982SVTEQDSK.,sp|P01834|IGKC_HUMAN,"255,890,500","212,915,250","64,215,000","162,854,250","127,326,000","268,752,500","258,960,000","281,722,500",...,"391,183,333","420,325,000","481,067,500","590,280,000","602,140,000","516,605,000","528,785,000","428,280,000","527,342,500",94


In [51]:
grouped["Peptidoform"]

In [56]:
signif_peptidoformes = [
    ".EI-14.016VLTQSPGTLSLSPGER.",
    ".AKLPD+14.016ATPTELAK.",
    ".LLIYDAS+60.036NR.",
    ".YVLTQPPSVSVAPGQTAR-28.006.",
    ".+57.021GPSVFPLAPSSK.",
    ".EVQLVQSGA+26.016EVK.",
    ".SGQAPVLVIYEDS+14.016K.",
    ".RPC+57.021FSA+30.011LEVDETYVPK.",
    ".YVTSAPM+15.995PEPQAPGR.",
    ".+43.006C+58.005TESLVNR."
]

In [70]:
count = 0
all_significant_proteins = []
for group_name, group_data in grouped["Peptidoform"]:
    # Find which values in the group are in signif_peptidoformes
    matching_values = group_data[group_data.isin(signif_peptidoformes)]
    
    if not matching_values.empty:
        count += 1
        print(f"Group Protein: {group_name}")
        print("Matching Peptidoform(s): ", matching_values)
        print("----------")
        all_significant_proteins.append(group_name)

    # break


Group Protein: sp|A0A075B6K4|LV310_HUMAN
Matching Peptidoform(s):  3052    .SGQAPVLVIYEDS+14.016K.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN
Matching Peptidoform(s):  2348    .LLIYDAS+60.036NR.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN
Matching Peptidoform(s):  1704    .EVQLVQSGA+26.016EVK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01619|KV301_HUMAN
Matching Peptidoform(s):  116    .EI-14.016VLTQSPGTLSLSPGER.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN
Matching Peptidoform(s):  4748    .+57.021GPSVFPLAPSSK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN
Matching Peptidoform(s):  23    .YVTSAPM+15.995PEPQAPGR.
Name: Peptidofor

In [71]:
all_significant_proteins

['sp|A0A075B6K4|LV310_HUMAN',
 'sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN',
 'sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN',
 'sp|P01619|KV301_HUMAN',
 'sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
 'sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
 'sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;sp|P02768|ALBU_HUMAN;tr|A0A087WWT3|A0A087WWT3_HUMAN;tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN;tr|B7WNR0|B7WNR0_HUMAN;tr|C9JKR2|C9JKR2_HUMAN;tr|D6RHD5|D6RHD5_HUMAN;tr|H0YA55|H0YA55_HUMAN',
 'sp|P02774-2|VTDB_HUMAN;sp|P02774-3|VTDB_HUMAN;sp|P02774|VTDB_HUMAN;tr|D6RF35|D6RF35_HUMAN',
 'sp|P80748|LV302_HUMAN']

In [75]:
df[df["Proteins"] == 'sp|A0A075B6K4|LV310_HUMAN'].shape

(35, 321)

In [81]:
for i in range(len(all_significant_proteins)):
    filtered_df = df[df["Proteins"] == all_significant_proteins[i]]
    # file_name = f"{all_significant_proteins[i]}.csv"
    file_path = os.path.join(f"./data/filetered_proteins/{i+1}.csv")
    filtered_df.to_csv(file_path, index=True)
    print(f"Saved: {file_path}")
    print(filtered_df.shape)

Saved: ./data/filetered_proteins/1.csv
(35, 321)
Saved: ./data/filetered_proteins/2.csv
(84, 321)
Saved: ./data/filetered_proteins/3.csv
(66, 321)
Saved: ./data/filetered_proteins/4.csv
(177, 321)
Saved: ./data/filetered_proteins/5.csv
(316, 321)
Saved: ./data/filetered_proteins/6.csv
(195, 321)
Saved: ./data/filetered_proteins/7.csv
(4420, 321)
Saved: ./data/filetered_proteins/8.csv
(59, 321)
Saved: ./data/filetered_proteins/9.csv
(143, 321)


In [210]:
df = pd.read_csv("./data/filetered_proteins/1.csv")
df.head()

,Unnamed: 0,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,3052,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052
1,6560,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6560
2,7078,.SGQAPVLVIY-15.995EDSK.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7078
3,7236,.SGQAPVLVIY-75.10EDSKRPSGIPER.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7236
4,7583,.SGQAPVLVIYEDSK+28.006RPSGIPER.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7583


In [211]:
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Columns: 322 entries, Unnamed: 0 to id
dtypes: float64(56), int64(2), object(264)
memory usage: 88.2+ KB


(   Unnamed: 0                      Peptidoform                   Proteins  \
 0        3052          .SGQAPVLVIYEDS+14.016K.  sp|A0A075B6K4|LV310_HUMAN   
 1        6560          .SGQAPVLVIYEDS+27.011K.  sp|A0A075B6K4|LV310_HUMAN   
 2        7078          .SGQAPVLVIY-15.995EDSK.  sp|A0A075B6K4|LV310_HUMAN   
 3        7236   .SGQAPVLVIY-75.10EDSKRPSGIPER.  sp|A0A075B6K4|LV310_HUMAN   
 4        7583  .SGQAPVLVIYEDSK+28.006RPSGIPER.  sp|A0A075B6K4|LV310_HUMAN   
 
   _dyn_#Patient_01.Timepoint_1  _dyn_#Patient_01.Timepoint_2  \
 0                          NaN                           NaN   
 1                          NaN                           NaN   
 2                          NaN                           NaN   
 3                          NaN                           NaN   
 4                          NaN                           NaN   
 
   _dyn_#Patient_01.Timepoint_3 _dyn_#Patient_01.Timepoint_4  \
 0                          NaN                          NaN   
 1        

In [212]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [213]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"38,405,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052
1,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6560
2,.SGQAPVLVIY-15.995EDSK.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7078
3,.SGQAPVLVIY-75.10EDSKRPSGIPER.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7236
4,.SGQAPVLVIYEDSK+28.006RPSGIPER.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7583


In [214]:
threshold = 0.9  # 90% threshold
max_na = df.shape[1] * threshold

df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)


In [215]:
df.shape

(4, 321)

In [216]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"38,405,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052
1,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6560
2,.FSGSSSGTM-2.946.,sp|A0A075B6K4|LV310_HUMAN,"51,704,000",NaN,"13,595,000","34,105,500","11,579,000","57,172,500","27,423,000",NaN,...,"42,235,000","38,735,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8614
3,.SGQAPVLVIYE-0.948.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"32,741,500",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14359


In [217]:
object_cols = df.select_dtypes(include='object').columns

# Convert comma-separated numeric strings to actual numbers
for col in object_cols:
    if col not in ['Peptidoform', 'Proteins']:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

df.head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38405000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3052
1,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6560
2,.FSGSSSGTM-2.946.,sp|A0A075B6K4|LV310_HUMAN,51704000.0,NaN,13595000.0,34105500.0,11579000.0,57172500.0,27423000.0,NaN,...,42235000.0,38735000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8614
3,.SGQAPVLVIYE-0.948.,sp|A0A075B6K4|LV310_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32741500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14359


In [218]:

timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]


for index, row in df.iterrows():
    for col in timepoints:
        if pd.isna(row[col]):
            if df[col].dtype in ['float64', 'int64']:
                median_val = row[timepoints].median(skipna=True)
                df.at[index, col] = median_val
            elif df[col].dtype == 'object':
                mode_val = row[timepoints].mode(dropna=True)
                if not mode_val.empty:
                    df.at[index, col] = mode_val[0]



In [219]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,38405000.0,...,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,65315500.0,3052
1,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,...,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,22944700.0,6560
2,.FSGSSSGTM-2.946.,sp|A0A075B6K4|LV310_HUMAN,51704000.0,17159000.0,13595000.0,34105500.0,11579000.0,57172500.0,27423000.0,17159000.0,...,42235000.0,38735000.0,17159000.0,17159000.0,17159000.0,17159000.0,17159000.0,17159000.0,17159000.0,8614
3,.SGQAPVLVIYE-0.948.,sp|A0A075B6K4|LV310_HUMAN,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,32741500.0,...,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,56124750.0,14359


In [220]:
patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
df[patient_cols] = np.log2(df[patient_cols] + 1)

In [221]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.194791,...,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,25.960922,3052
1,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,...,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,24.451658,6560
2,.FSGSSSGTM-2.946.,sp|A0A075B6K4|LV310_HUMAN,25.623773,24.032462,23.696573,25.023501,23.465007,25.768818,24.708883,24.032462,...,25.331936,25.207134,24.032462,24.032462,24.032462,24.032462,24.032462,24.032462,24.032462,8614
3,.SGQAPVLVIYE-0.948.,sp|A0A075B6K4|LV310_HUMAN,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,24.964617,...,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,25.742134,14359


In [222]:

patient_timepoints = defaultdict(list)
for col in patient_cols:
    parts = col.split('.Timepoint_')
    if len(parts) == 2:
        patient_id = parts[0]
        patient_timepoints[patient_id].append(col)

differences = pd.DataFrame()
for patient_id, cols in patient_timepoints.items():
    sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
    first_col = sorted_cols[0]
    last_col = sorted_cols[-1]
    diff_col_name = f"{patient_id}_range"
    differences[diff_col_name] = df[last_col] - df[first_col]


In [223]:
# differences['Peptidoform'] = df['Peptidoform']
# differences['Proteins'] = df['Proteins']
print(differences.shape)
differences.head()

(4, 52)


,_dyn_#Patient_01_range,_dyn_#Patient_02_range,_dyn_#Patient_03_range,_dyn_#Patient_05_range,_dyn_#Patient_06_range,_dyn_#Patient_07_range,_dyn_#Patient_08_range,_dyn_#Patient_09_range,_dyn_#Patient_10_range,_dyn_#Patient_11_range,...,_dyn_#Patient_47_range,_dyn_#Patient_48_range,_dyn_#Patient_49_range,_dyn_#Patient_50_range,_dyn_#Patient_51_range,_dyn_#Patient_52_range,_dyn_#Patient_53_range,_dyn_#Patient_54_range,_dyn_#Patient_57_range,_dyn_#Patient_58_range
0,0.000000,-0.204704,0.000000,0.0,0.000000,0.000000,-1.407899,0.0,0.0,1.115728,...,0.518783,0.0,0.00000,0.0,-0.058745,0.0,0.000000,0.0,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,-0.914889,0.000000,0.612242,0.0,0.611993,-1.728164,0.000000,0.0,0.0,-1.999424,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.471254,0.0
3,0.000000,-0.730754,0.000000,0.0,0.000000,0.227135,-0.671581,0.0,0.0,0.000000,...,0.000000,0.0,0.46046,0.0,0.000000,0.0,0.424168,0.0,0.000000,0.0


0   -0.025083
1    0.003457
2   -0.049458
3   -0.003383
dtype: float64

In [208]:
values = differences.mean(axis=1)
values

0   -0.015211
1   -0.011324
2    0.032630
3   -0.163438
4   -0.097236
5   -0.275318
6   -0.020481
7    0.060674
8    0.014177
9   -0.029495
dtype: float64

In [ ]:
file_path = os.path.join(f"./data/filetered_proteins/{+1}.csv")
filtered_df.to_csv(file_path, index=True)

In [225]:
rootDir = './data/filetered_proteins/'
for filename in os.listdir(rootDir):
    df = pd.read_csv(rootDir+filename)
    # print(df.head())
    df.drop('Unnamed: 0', axis=1, inplace=True)
    print(f"Data Shape for {filename} is {df.shape}")
    threshold = 0.9  # 90% threshold
    max_na = df.shape[1] * threshold

    df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)
    print(f"Data Shape for {filename} after dropping 90% data {df.shape}")
    object_cols = df.select_dtypes(include='object').columns

    # Convert comma-separated numeric strings to actual numbers
    for col in object_cols:
        if col not in ['Peptidoform', 'Proteins']:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')


    timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]


    for index, row in df.iterrows():
        for col in timepoints:
            if pd.isna(row[col]):
                if df[col].dtype in ['float64', 'int64']:
                    median_val = row[timepoints].median(skipna=True)
                    df.at[index, col] = median_val
                elif df[col].dtype == 'object':
                    mode_val = row[timepoints].mode(dropna=True)
                    if not mode_val.empty:
                        df.at[index, col] = mode_val[0]

    patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
    df[patient_cols] = np.log2(df[patient_cols] + 1)

    patient_timepoints = defaultdict(list)
    for col in patient_cols:
        parts = col.split('.Timepoint_')
        if len(parts) == 2:
            patient_id = parts[0]
            patient_timepoints[patient_id].append(col)

    differences = pd.DataFrame()
    for patient_id, cols in patient_timepoints.items():
        sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
        first_col = sorted_cols[0]
        last_col = sorted_cols[-1]
        diff_col_name = f"{patient_id}_range"
        differences[diff_col_name] = df[last_col] - df[first_col]

    row_means = differences.mean(axis=1)
    differences["avg"] = row_means
    differences['Peptidoform'] = df['Peptidoform']
    differences['Proteins'] = df['Proteins']


    

    file_path = os.path.join(f"./data/log_data/{filename}")
    differences.to_csv(file_path, index=True)   
        


Data Shape for 6.csv is (195, 321)
Data Shape for 6.csv after dropping 90% data (46, 321)


/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_11977/3758358975.py:3: DtypeWarning: Columns (3,4,5,6,7,9,13,24,26,27,28,29,30,54,55,59,60,61,62,63,64,65,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,116,117,118,119,120,130,132,135,144,152,153,154,155,156,157,158,159,173,174,175,176,177,178,188,191,192,193,194,195,196,197,200,201,202,203,204,205,206,214,215,216,217,218,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,242,244,245,247,248,250,290,291,292,293,296,298,299,301,302,307,312,313,314,315,316,317,318,319,320) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(rootDir+filename)


Data Shape for 7.csv is (4420, 321)
Data Shape for 7.csv after dropping 90% data (375, 321)
Data Shape for 5.csv is (316, 321)
Data Shape for 5.csv after dropping 90% data (37, 321)
Data Shape for 4.csv is (177, 321)
Data Shape for 4.csv after dropping 90% data (41, 321)
Data Shape for 1.csv is (35, 321)
Data Shape for 1.csv after dropping 90% data (4, 321)
Data Shape for 3.csv is (66, 321)
Data Shape for 3.csv after dropping 90% data (25, 321)
Data Shape for 2.csv is (84, 321)
Data Shape for 2.csv after dropping 90% data (24, 321)
Data Shape for 9.csv is (143, 321)
Data Shape for 9.csv after dropping 90% data (23, 321)
Data Shape for 8.csv is (59, 321)
Data Shape for 8.csv after dropping 90% data (10, 321)


,Unnamed: 0,_dyn_#Patient_01_range,_dyn_#Patient_02_range,_dyn_#Patient_03_range,_dyn_#Patient_05_range,_dyn_#Patient_06_range,_dyn_#Patient_07_range,_dyn_#Patient_08_range,_dyn_#Patient_09_range,_dyn_#Patient_10_range,...,_dyn_#Patient_50_range,_dyn_#Patient_51_range,_dyn_#Patient_52_range,_dyn_#Patient_53_range,_dyn_#Patient_54_range,_dyn_#Patient_57_range,_dyn_#Patient_58_range,avg,Peptidoform,Proteins
0,0,0.000000,-0.204704,0.000000,0.0,0.000000,0.000000,-1.407899,0.0,0.0,...,0.0,-0.058745,0.0,0.000000,0.0,0.000000,0.0,-0.025083,.SGQAPVLVIYEDS+14.016K.,sp|A0A075B6K4|LV310_HUMAN
1,1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.003457,.SGQAPVLVIYEDS+27.011K.,sp|A0A075B6K4|LV310_HUMAN
2,2,-0.914889,0.000000,0.612242,0.0,0.611993,-1.728164,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.471254,0.0,-0.049458,.FSGSSSGTM-2.946.,sp|A0A075B6K4|LV310_HUMAN
3,3,0.000000,-0.730754,0.000000,0.0,0.000000,0.227135,-0.671581,0.0,0.0,...,0.0,0.000000,0.0,0.424168,0.0,0.000000,0.0,-0.003383,.SGQAPVLVIYE-0.948.,sp|A0A075B6K4|LV310_HUMAN
